In [1]:
from openml import tasks

from sklearn.model_selection import train_test_split

from classes import WeightClipper
from functions import run_eagga_cv

from tqdm import tqdm

In [2]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)
oml_tasks

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


[OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 37
 Task URL.............: https://www.openml.org/t/37
 Estimation Procedure.: crossvalidation
 Target Feature.......: class
 # of Classes.........: 2
 Cost Matrix..........: Available,
 OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 43
 Task URL.............: https://www.openml.org/t/43
 Estimation Procedure.: crossvalidation
 Target Feature.......: class
 # of Classes.........: 2
 Cost Matrix..........: Available,
 OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 3903
 Task URL.............: https://www.openml.org/t/3903
 Estimation Procedure.: crossvalidation
 Target Feature.......: c
 # of Classes.........: 2
 Cost Matrix..........: Available,
 OpenML Classification Task
 Ta

In [3]:
oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

class_column_in_X = list()
empty_y = list()
classes = list()
names = list()
for oml_dataset in tqdm(oml_datasets):
    class_column = oml_dataset.default_target_attribute
    X, y, _, _ = oml_dataset.get_data()

    class_column_in_X.append(class_column in X.columns)
    empty_y.append(y is None)

    names.append(oml_dataset.name)
    classes.append(X.loc[:, class_column].unique().tolist())
print(f'{all(class_column_in_X)}: {class_column_in_X}')  # X always already includes y
print(f'{all(empty_y)}: {empty_y}')  # y is always None

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 23.38it/s]

True: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
True: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


we see: X always includes the target and y is always None for our datasets

In [4]:
# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']
for name, cs, class_pos in zip(names, classes, positive_classes):
    print(f'{name}: {cs}, {class_pos}')

diabetes: ['tested_positive', 'tested_negative'], tested_positive
spambase: ['1', '0'], 1
pc3: [False, True], True
jm1: [False, True], True
kc2: ['no', 'yes'], yes
pc1: [False, True], True
banknote-authentication: ['1', '2'], 2
wdbc: ['2', '1'], 2
climate-model-simulation-crashes: ['0', '1'], 1
blood-transfusion-service-center: ['2', '1'], 2
gina: ['0', '1'], 1
kc1: [False, True], True
madeline: ['1', '0'], 1
numerai28.6: ['0', '1'], 1
ozone-level-8hr: ['1', '2'], 2
philippine: ['0', '1'], 1
phoneme: ['1', '2'], 2
Satellite: ['Anomaly', 'Normal'], Anomaly
sylvine: ['1', '0'], 1
wilt: ['2', '1'], 2


In [5]:
oml_task_diabetes = oml_tasks[0]
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()
Xy = X.copy()
Xy

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [6]:
Xy.loc[:, 'class'].nunique() == 1

False

In [7]:
tmp, _, _, _ = tasks.get_task(3904).get_dataset().get_data()
tmp.isnull().values.any()

np.True_

In [10]:
cv_k = 5

mu = 3  # TODO: set to 100
la = 2  # TODO: set to 10

patience = 2  # TODO: set higher, e.g. 10
secs_per_fold = 1
secs_total = 30  # also breaks mid-generation, will only finish current individual in case of breach

monotonicity_clipper = WeightClipper(0, None)  # enforce monotonicity by clipping weights to [0, infty) after each epoch (in def train)

for i, (oml_dataset, class_positive) in enumerate(zip(oml_datasets, positive_classes)):
    print(f'Running EAGGA-NN on {oml_dataset.name} (id {oml_dataset.id}) dataset')
    Xy, _, categorical_indicator, _ = oml_dataset.get_data()
    class_column = oml_dataset.default_target_attribute

    if i == 3:
        print(f'drop {Xy.loc[Xy.isnull().any(axis=1), :].shape[0]} NaN values of jm1 dataset')
        Xy = Xy.dropna(axis=0, how='any')
        Xy = Xy.reset_index(drop=True)

    if Xy.isnull().values.any():
        print(f'contains some NaN values, {Xy.loc[Xy.isnull().any(axis=1), :]}')
    if any(Xy.isnull().all(axis=0).values):
        print(f'contains entire NaN column(s): {Xy.isnull().all(axis=0)}')
    if any(Xy.isnull().all(axis=1).values):
        print(f'contains entire NaN row(s): {Xy.isnull().all(axis=1)}')

    # outer split, for holdout
    data_train_val, data_test = train_test_split(
        Xy,
        train_size=2/3,
        shuffle=True,
        stratify=Xy.loc[:, class_column]
    )
    # reset indices as StratifiedKFold assumes consecutive index
    data_train_val = data_train_val.reset_index(drop=True)
    data_test = data_test.reset_index(drop=True)

    '''
    - for each individual in the configuration, run k (inner) folds + average its performance
        -> in each fold, additionally split the training data, train on larger split, use smaller split to determine early stopping
        -> average early stopping epoch over all folds, report back with average performance
    - find pareto front
    - evaluate pareto front's performance on holdout test set (data_val), each model of the front is trained for the average of the epochs determined by early stopping in CV
    '''
    run_eagga_cv(mu, la, cv_k, data_train_val, categorical_indicator, class_column, class_positive, epochs=10, batch_size=8, patience=patience, weight_clipper=monotonicity_clipper, secs_per_fold=secs_per_fold, secs_total=secs_total)

Running EAGGA-NN on ozone-level-8hr (id 1487) dataset


f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Start EA at 2025-02-26T00:42:14.398403
Evolution 1, evaluate 3 individuals
Running HPO for individual 1/3: 6 total layers, 4 nodes per hidden layer, gs: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71], [[[64], 0]])
Fold 1/5 | (0.6436082319363076, 0.013888888888888888, 0.0, 0.013888888888888888) 3
Fold 2/5 | (0.7034700315457412, 0.013888888888888888, 0.0, 0.013888888888888888) 3
Fold 3/5 | (0.768915420023015, 0.013888888888888888, 0.0, 0.013888888888888888) 3
Fold 4/5 | (0.5, 0.013888888888888888, 0.0, 0.013888888888888888) 3
Fold 5/5 | (0.5, 0.013888888888888888, 0.0, 0.013888888888888888) 3
Running HPO for individual 2/3: 5 total layers, 6 nodes per hidden layer, gs: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


Start EA at 2025-02-26T00:48:29.011627
Evolution 1, evaluate 3 individuals
Running HPO for individual 1/3: 4 total layers, 3 nodes per hidden layer, gs: ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 19

...

(0.7179675788415163, 0.6, 0.1, 0.6) 1
Fold 5/5 | 

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\torch\nn\init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(0.6574549111258019, 0.6, 0.1, 0.6) 1
Finished EA at 2025-02-26T00:49:37.920858
Running EAGGA-NN on Satellite (id 40900) dataset
Start EA at 2025-02-26T00:49:57.896108
Evolution 1, evaluate 3 individuals
Running HPO for individual 1/3: 6 total layers, 3 nodes per hidden layer, gs: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [[[12], 0]])
Fold 1/5 | (0.5, 0.027777777777777776, 0.0, 0.027777777777777776) 2
Fold 2/5 | (0.5, 0.027777777777777776, 0.0, 0.027777777777777776) 2
Fold 3/5 | (0.5, 0.027777777777777776, 0.0, 0.027777777777777776) 2
Fold 4/5 | (0.5, 0.027777777777777776, 0.0, 0.027777777777777776) 2
Fold 5/5 | (0.5, 0.027777777777777776, 0.0, 0.027777777777777776) 2
Running HPO for individual 2/3: 5 total layers, 3 nodes per hidden layer, gs: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35], [[[32, 17], 1]])
Fold 1/5

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Start EA at 2025-02-26T00:50:50.381405
Evolution 1, evaluate 3 individuals
Running HPO for individual 1/3: 4 total layers, 3 nodes per hidden layer, gs: ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [[[0], 1]])
Fold 1/5 | (0.5069936048698744, 0.05, 0.0, 0.0) 2
Fold 2/5 | (0.5, 0.05, 0.0, 0.0) 2
Fold 3/5 | (0.5015648848416251, 0.05, 0.0, 0.0) 2
Fold 4/5 | (0.5423762240400611, 0.05, 0.0, 0.0) 2
Fold 5/5 | (0.47172060160175605, 0.05, 0.0, 0.0) 2
Running HPO for individual 2/3: 3 total layers, 5 nodes per hidden layer, gs: ([0, 1, 2, 3, 4, 5, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19], [[[8, 6], 0], [[12], 0]])
Fold 1/5 | (0.7776580828289046, 0.15, 0.005263157894736842, 0.15) 2
Fold 2/5 | (0.620281765018607, 0.15, 0.005263157894736842, 0.15) 2
Fold 3/5 | (0.7600366997650528, 0.15, 0.005263157894736842, 0.15) 2
Fold 4/5 | (0.7132959475913636, 0.15, 0.005263157894736842, 0.15) 2
Fold 5/5 | (0.5948620328925932, 0.15, 0.005263157894736842, 0.15) 2
Running HPO for individ